In [0]:
df_goalscorers = spark.read.table('hive_metastore.default.goalscorers')
df_results = spark.read.table('hive_metastore.default.results')

In [0]:
df_goalscorers.head()
df_results.head()

Row(date=datetime.date(1872, 11, 30), home_team='Scotland', away_team='England', home_score=0, away_score=0, tournament='Friendly', city='Glasgow', country='Scotland', neutral=False)

In [0]:
## Number of goals scored by Lionel Messi
df_goalscorers.createOrReplaceTempView("goalscorers")
spark.sql("SELECT scorer, COUNT(*) FROM goalscorers WHERE scorer = 'Lionel Messi' AND own_goal = false GROUP BY scorer").show()

+------------+--------+
|      scorer|count(1)|
+------------+--------+
|Lionel Messi|      55|
+------------+--------+



Llistat dels 5 partits més recents que ha jugat la selecció espanyola.

In [0]:
df_results.createOrReplaceTempView("goalscorers")
spark.sql("SELECT * FROM goalscorers WHERE home_team = 'Spain' OR away_team = 'Spain' ORDER BY date DESC LIMIT 5").show()

+----------+-----------+-----------+----------+----------+-------------------+--------------------+-----------+-------+
|      date|  home_team|  away_team|home_score|away_score|         tournament|                city|    country|neutral|
+----------+-----------+-----------+----------+----------+-------------------+--------------------+-----------+-------+
|2025-03-23|      Spain|Netherlands|         3|         3|UEFA Nations League|            Valencia|      Spain|  false|
|2025-03-20|Netherlands|      Spain|         2|         2|UEFA Nations League|           Rotterdam|Netherlands|  false|
|2024-11-18|      Spain|Switzerland|         3|         2|UEFA Nations League|Santa Cruz de Ten...|      Spain|  false|
|2024-11-15|    Denmark|      Spain|         1|         2|UEFA Nations League|          Copenhagen|    Denmark|  false|
|2024-10-15|      Spain|     Serbia|         3|         0|UEFA Nations League|             Cordoba|      Spain|  false|
+----------+-----------+-----------+----

Nombre de gols que ha marcat Espanya en tota la seva història. Aquesta informació s'ha de treure de results, ja que goalscorers no conté tots els gols.

In [0]:
df_results.createOrReplaceTempView("results")
spark.sql("SELECT SUM(CASE WHEN home_team = 'Spain' THEN home_score ELSE 0 end) + SUM(CASE WHEN away_team = 'Spain' THEN away_score ELSE 0 end) as total_goals FROM results").show()

+-----------+
|total_goals|
+-----------+
|       1567|
+-----------+



Llistat dels 5 màxims golejadors amb la selecció espanyola (sense comptar autogols).

In [0]:
df_goalscorers.createOrReplaceGlobalTempView('goalscorers')
spark.sql("SELECT scorer, count(*) AS goals FROM goalscorers WHERE team = 'Spain' AND own_goal = false GROUP BY scorer ORDER BY goals DESC LIMIT 5").show()

+---------------+-----+
|         scorer|goals|
+---------------+-----+
|    David Villa|   41|
|           Raúl|   32|
|  Álvaro Morata|   29|
|Fernando Torres|   28|
|Fernando Hierro|   25|
+---------------+-----+



Llistat dels jugadors espanyols que han marcat algun gol de penal en alguna Eurocopa (UEFA Euro), ordenats alfabèticament.

In [0]:
df_goalscorers.createOrReplaceTempView("goalscorers")
df_results.createOrReplaceTempView("results")
spark.sql("SELECT DISTINCT scorer from results r INNER JOIN goalscorers g on r.date = g.date AND r.home_team = g.home_team AND r.away_team = g.away_team WHERE g.team = 'Spain' AND g.penalty = true AND r.tournament like '%Euro%' ORDER BY scorer").show()

+--------------------+
|              scorer|
+--------------------+
|      Andrés Iniesta|
|         Daniel Ruiz|
|         David Villa|
|     Fernando Hierro|
|Francisco José Ca...|
|     Gaizka Mendieta|
|      José Claramunt|
|  Juan Antonio Señor|
|              Míchel|
|               Pirri|
|        Sergio Ramos|
|         Xabi Alonso|
|       Álvaro Morata|
+--------------------+



Llistat dels 5 màxims golejadors de les fases finals dels mundials (FIFA World Cup) (sense comptar autogols).

In [0]:
df_goalscorers.createOrReplaceTempView('goalscorers')
df_results.createOrReplaceTempView('reults')
spark.sql('''
    SELECT scorer, COUNT(*) AS goals
    FROM results r
    JOIN goalscorers g
    ON r.date = g.date AND r.home_team = g.home_team AND r.away_team = g.away_team
    WHERE r.tournament = 'FIFA World Cup' AND g.own_goal = 'FALSE'
    GROUP BY scorer
    ORDER BY goals DESC
    LIMIT 5
''').show()

+--------------+-----+
|        scorer|goals|
+--------------+-----+
|Miroslav Klose|   16|
|       Ronaldo|   15|
|   Gerd Müller|   14|
|  Lionel Messi|   13|
| Just Fontaine|   13|
+--------------+-----+

